Vest - verified parsers generated

https://mars-research.github.io/projects/atmo/
NPL nested permission language.
A linear ghost map of permissions + Raw pointers. Somehow this works well? maps are maintained disjoint somehow? I guess linear keys must be disjoint if putting them in map consumes them. Hmm.


can I apply verus style something to assembly?
Can I have linearity at meta level in knuckeldragger?
"derivative is constant" ?
https://ncatlab.org/nlab/files/Zdancewic-LinearLogicAlgebra.pdf

"Depedent type" judgements get translated in a different way.


Linear dafny

compare to
creusot
aeneas
flux
prusti

https://verus-lang.github.io/event-sites/2024-sosp/



In [2]:
%%file /tmp/myverus.rs
use vstd::prelude::*;

verus! {

spec fn min(x: int, y: int) -> int {
    if x <= y {
        x
    } else {
        y
    }
}

fn main() {
    assert(min(10, 20) == 10);
    assert(min(-10, -20) == -20);
    assert(forall|i: int, j: int| min(i, j) <= i && min(i, j) <= j);
    assert(forall|i: int, j: int| min(i, j) == i || min(i, j) == j);
    assert(forall|i: int, j: int| min(i, j) == min(j, i));
}

} // verus!

Writing /tmp/myverus.rs


In [3]:
! ~/Downloads/verus/verus /tmp/myverus.rs

verification results:: 1 verified, 0 errors
